# Spherepop Reference Implementation (Pedagogical Kernel)

This notebook implements a minimal, deterministic Spherepop OS-style kernel
with an append-only event log and replay semantics.

Authoritative event types:

- POP
- MERGE
- LINK
- UNLINK
- COLLAPSE
- LABEL

LABEL is metadata-only and does not affect semantic structure.


## 1. Event Definitions

We represent events as simple dictionaries for clarity.


In [ ]:
from dataclasses import dataclass
from typing import Dict, Tuple, List, Set
from collections import defaultdict

@dataclass(frozen=True)
class Event:
    type: str
    args: tuple


## 2. Union–Find for Equivalence (MERGE + COLLAPSE)

This maintains canonical representatives.


In [ ]:
class UnionFind:
    def __init__(self):
        self.parent = {}

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def add(self, x):
        if x not in self.parent:
            self.parent[x] = x

    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra


## 3. Kernel State

σ = (O, U, R, M)

O: objects  
U: equivalence structure  
R: typed relations  
M: metadata map  


In [ ]:
class Kernel:
    def __init__(self):
        self.objects: Set[str] = set()
        self.uf = UnionFind()
        self.relations: Set[Tuple[str, str, str]] = set()
        self.meta: Dict[str, Dict[str, str]] = defaultdict(dict)
        self.log: List[Event] = []

    def apply(self, event: Event):
        t = event.type
        args = event.args

        if t == "POP":
            o = args[0]
            self.objects.add(o)
            self.uf.add(o)

        elif t == "MERGE":
            a, b = args
            self.uf.union(a, b)

        elif t == "LINK":
            a, b, rel = args
            ra, rb = self.uf.find(a), self.uf.find(b)
            self.relations.add((ra, rb, rel))

        elif t == "UNLINK":
            a, b, rel = args
            ra, rb = self.uf.find(a), self.uf.find(b)
            self.relations.discard((ra, rb, rel))

        elif t == "COLLAPSE":
            objs = args[0]
            root = objs[0]
            for o in objs[1:]:
                self.uf.union(root, o)

        elif t == "LABEL":
            o, key, value = args
            self.meta[o][key] = value

        else:
            raise ValueError(f"Unknown event type: {t}")

    def append(self, event: Event):
        self.log.append(event)
        self.apply(event)

    def replay(self):
        self.__init__()
        for e in self.log:
            self.apply(e)

    def snapshot(self):
        return {
            "objects": list(self.objects),
            "representatives": {o: self.uf.find(o) for o in self.objects},
            "relations": list(self.relations),
            "metadata": dict(self.meta)
        }


## 4. Example Usage


In [ ]:
kernel = Kernel()

kernel.append(Event("POP", ("A",)))
kernel.append(Event("POP", ("B",)))
kernel.append(Event("LINK", ("A", "B", "connected")))
kernel.append(Event("LABEL", ("A", "name", "Root node")))
kernel.append(Event("MERGE", ("A", "B")))

kernel.snapshot()


## 5. Deterministic Replay Validation

Spherepop state must be fully determined by the event log.
We validate this by snapshotting, replaying from scratch,
and comparing results.


In [ ]:
snapshot_before = kernel.snapshot()

kernel.replay()

snapshot_after = kernel.snapshot()

snapshot_before == snapshot_after


## 6. Event-by-Event Replay Trace

This cell replays the log step by step and records
intermediate snapshots to demonstrate causal accumulation.


In [ ]:
def replay_trace(log):
    k = Kernel()
    trace = []
    for e in log:
        k.apply(e)
        trace.append((e, k.snapshot()))
    return trace

trace = replay_trace(kernel.log)

for event, state in trace:
    print("Event:", event)
    print("State:", state)
    print("-" * 40)


## 7. Relation Graph Visualization

We visualize representative-normalized relations
as a simple directed graph.


In [ ]:
import matplotlib.pyplot as plt

def draw_graph(snapshot):
    objects = snapshot["objects"]
    relations = snapshot["relations"]

    positions = {}
    for i, obj in enumerate(objects):
        positions[obj] = (i, 0)

    for obj in objects:
        plt.scatter(*positions[obj])
        plt.text(positions[obj][0], positions[obj][1] + 0.1, obj)

    for a, b, rel in relations:
        x1, y1 = positions[a]
        x2, y2 = positions[b]
        plt.plot([x1, x2], [y1, y2])
        plt.text((x1 + x2)/2, (y1 + y2)/2, rel)

    plt.title("Spherepop Relation Graph")
    plt.yticks([])
    plt.show()

draw_graph(kernel.snapshot())


## 8. Speculative Overlay (Non-Authoritative Branch)

This demonstrates how one could simulate a speculative branch
without committing events to the authoritative log.


In [ ]:
overlay_kernel = Kernel()
overlay_kernel.log = list(kernel.log)

overlay_kernel.apply(Event("POP", ("C",)))
overlay_kernel.apply(Event("LINK", ("C", "A", "proposed")))

print("Authoritative state:")
print(kernel.snapshot())

print("\nOverlay state:")
print(overlay_kernel.snapshot())


## 9. COLLAPSE Demonstration

COLLAPSE merges multiple objects into a single representative.


In [ ]:
collapse_kernel = Kernel()

collapse_kernel.append(Event("POP", ("X",)))
collapse_kernel.append(Event("POP", ("Y",)))
collapse_kernel.append(Event("POP", ("Z",)))

collapse_kernel.append(Event("COLLAPSE", (["X", "Y", "Z"],)))

collapse_kernel.snapshot()
